<a href="https://colab.research.google.com/github/NEY-DPI/Vibrations/blob/main/Vibrations_KWN52.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Import libraries, functions

In [ ]:
#@title Import python libraries
import sys

import pandas as pd
import math as m
import ipywidgets as widgets

pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [ ]:
#@title Functions libary:
def get_freq(path):
    df = pd.read_csv(path)
    df.columns = ['LC', 'Title', 'f']
    return df


def check_freq(df_in):
    df = df_in
    f_min = df_in.f.min()
    if f_min > 5:
        df['vert_to_check'] = False
    else:
        vert = (1.7 < df.f) & (df.f < 2.2)
        df['vert_to_check'] = vert

    hor = (0.3 < df.f) & (df.f < 2.5)
    df['hor_to_check'] = hor

    return df


def get_nodes(path):
    df = pd.read_csv(path)
    df.columns = ['NODE', 'X', 'Y', 'Z']
    return df


def get_Si(path):
    df = pd.read_csv(path)
    df.columns = ['NODE', 'Si']
    df['Si'] = abs(df['Si'])
    return df


def merge_nodes(nodes, Si):
    df = pd.merge(nodes, Si, on='NODE')
    return df


def get_modal_disp(path):
    df = pd.read_csv(path, low_memory=False)
    df.columns = ['LC', 'Title', 'NODE', 'ux', 'uy', 'uz']
    return df


def scale_modal_disp(modal_disp, factor):
    df = modal_disp
    df['sc_ux'] = modal_disp['ux'] * factor
    df['sc_uy'] = modal_disp['uy'] * factor
    df['sc_uz'] = modal_disp['uz'] * factor
    return df


def merge_all_mode(displ, nodes):
    df = pd.merge(nodes, displ, on='NODE')
    return df


def get_axis(dir, vertical, longitudinal, lateral):
    if vertical == 'x':
        vertical = 'sc_ux'
    elif vertical == 'y':
        vertical = 'sc_uy'
    elif vertical == 'z':
        vertical = 'sc_uz'
    if longitudinal == 'x':
        longitudinal = 'sc_ux'
    elif longitudinal == 'y':
        longitudinal = 'sc_uy'
    elif longitudinal == 'z':
        longitudinal = 'sc_uz'
    if lateral == 'x':
        lateral = 'sc_ux'
    elif lateral == 'y':
        lateral = 'sc_uy'
    elif lateral == 'z':
        lateral = 'sc_uz'

    if dir == 'vertical':
        return vertical
    elif dir == 'longitudinal':
        return longitudinal
    elif dir == 'lateral':
        return lateral


def get_d(TC, S):
    if TC == 1:
        d = 15 / S
    elif TC == 2:
        d = 0.2
    elif TC == 3:
        d = 0.5
    elif TC == 4:
        d = 1
    return d


def get_modal_mass(disp, axis):
    max_disp_all = max(abs(disp[axis].max()),
                       abs(disp[axis].min())
                       )
    modal_mass = 1 / (max_disp_all ** 2)
    return modal_mass


def get_damping(TC, bridge_type):
    values = pd.DataFrame(
        {'type': ['Reinforced Concrete',
                  'Prestressed Concrete',
                  'Steel-Concrete',
                  'Steel',
                  'Wood'],
         '<1p/m2': [0.008, 0.005, 0.003, 0.002, 0.010],
         '>=1p/m2': [0.016, 0.010, 0.006, 0.004, 0.020]
         })
    if TC == 4:
        col = '>=1p/m2'
    else:
        col = '<1p/m2'
    return values[values.type == bridge_type][col].item()


def get_N_eq(N, damping, d):
    if d < 1:
        return 10.8 * m.sqrt(damping * N)
    else:
        return 1.85 * m.sqrt(N)


def get_gp(f, direction, phase='design'):
    if direction == 'vertical':
        if phase == 'design':
            if f <= 0.5:
                gp = 0
            elif f <= 1.3:
                gp = 0 + (280 - 0) * (f - 0.5) / (1.3 - 0.5)
            elif f <= 2.4:
                gp = 280
            elif f <= 3:
                gp = 280 + (70 - 280) * (f - 2.4) / (3 - 2.4)
            elif f <= 4.8:
                gp = 70
            elif f <= 6:
                gp = 70 + (0 - 70) * (f - 4.8) / (6 - 4.8)
            else:
                gp = 0
        else:
            if f <= 1:
                gp = 0
            elif f <= 1.6:
                gp = 0 + (280 - 0) * (f - 1) / (1.6 - 1)
            elif f <= 2.1:
                gp = 280
            elif f <= 2.55:
                gp = 280 + (35 - 280) * (f - 2.1) / (2.55 - 2.1)
            elif f <= 2.95:
                gp = 35
            elif f <= 3.4:
                gp = 35 + (70 - 35) * (f - 2.95) / (3.4 - 2.95)
            elif f <= 4.25:
                gp = 70
            elif f <= 5:
                gp = 70 + (0 - 70) * (f - 4.25) / (5 - 4.25)
            else:
                gp = 0
    if direction == 'longitudinal':
        if phase == 'design':
            if f <= 0.5:
                gp = 0
            elif f <= 1.3:
                gp = 0 + (140 - 0) * (f - 0.5) / (1.3 - 0.5)
            elif f <= 2.4:
                gp = 140
            elif f <= 3:
                gp = 140 + (35 - 140) * (f - 2.4) / (3 - 2.4)
            elif f <= 4.8:
                gp = 35
            elif f <= 6:
                gp = 35 + (0 - 35) * (f - 4.8) / (6 - 4.8)
            else:
                gp = 0
        else:
            if f <= 1:
                gp = 0
            elif f <= 1.6:
                gp = 0 + (140 - 0) * (f - 1) / (1.6 - 1)
            elif f <= 2.1:
                gp = 140
            elif f <= 2.55:
                gp = 140 + (17.5 - 140) * (f - 2.1) / (2.55 - 2.1)
            elif f <= 2.95:
                gp = 17.5
            elif f <= 3.4:
                gp = 17.5 + (35 - 17.5) * (f - 2.95) / (3.4 - 2.95)
            elif f <= 4.25:
                gp = 35
            elif f <= 5:
                gp = 35 + (0 - 35) * (f - 4.25) / (5 - 4.25)
            else:
                gp = 0
    if direction == 'lateral':
        if phase == 'design':
            if f <= 0.15:
                gp = 0
            elif f <= 0.4:
                gp = 0 + (35 - 0) * (f - 0.15) / (0.4 - 0.15)
            elif f <= 1.2:
                gp = 35
            elif f <= 1.4:
                gp = 35 + (7 - 35) * (f - 1.2) / (1.4 - 1.2)
            elif f <= 2.4:
                gp = 7
            elif f <= 2.8:
                gp = 7 + (0 - 7) * (f - 2.4) / (2.8 - 2.4)
            else:
                gp = 0
        else:
            if f <= 0.3:
                gp = 0
            elif f <= 0.5:
                gp = 0 + (35 - 0) * (f - 0.3) / (0.5 - 0.3)
            elif f <= 1.1:
                gp = 35
            elif f <= 1.3:
                gp = 35 + (3.5 - 35) * (f - 1.1) / (1.3 - 1.1)
            elif f <= 1.45:
                gp = 3.5
            elif f <= 1.7:
                gp = 3.5 + (7 - 3.5) * (f - 1.45) / (1.7 - 1.45)
            elif f <= 2.1:
                gp = 7
            elif f <= 2.5:
                gp = 7 + (0 - 7) * (f - 2.1) / (2.5 - 2.1)
            else:
                gp = 0
    return gp


def get_a_req(direction, TC, traffic):
    if traffic == 'Dense':
        values = pd.DataFrame(
            {'TC': [1, 2, 3, 4],
             'vertical': [0.70, 1, 1, 2.5],
             'longitudinal': [0.2, 0.3, 0.3, 0.4],
             'lateral': [0.2, 0.3, 0.3, 0.4]
            }
        )
    else:
        values = pd.DataFrame(
            {'TC': [1, 2, 3, 4],
             'vertical': [0.70, 1, 2.5, 2.5],
             'longitudinal': [0.2, 0.3, 0.4, 0.4],
             'lateral': [0.2, 0.3, 0.4, 0.4]
            }
        )
    return values[values.TC == TC][direction].item()




# 2. Import files

In [ ]:
path_eigenfrequencies = "/content/drive/MyDrive/Colab Notebooks/Vibrations/KWN52/eigenfrequencies.csv"
path_modal_displacements = "/content/drive/MyDrive/Colab Notebooks/Vibrations/KWN52/modal-displacements.csv"
path_node_coordinates = "/content/drive/MyDrive/Colab Notebooks/Vibrations/KWN52/node-coordinates.csv"
path_unit_load_Si = "/content/drive/MyDrive/Colab Notebooks/Vibrations/KWN52/unit-load-Si.csv"

# 3. Check eigenfrequencies

In [ ]:
#@title Inputs
first_lc_empty_w = widgets.IntText(value=7500)
first_lc_loaded_w = widgets.IntText(value=7600)

left_box = widgets.VBox([
    widgets.Label('First LC of eigenfrequency - empty bridge:'),
    widgets.Label('First LC of eigenfrequency - loaded bridge:')
    ])

right_box = widgets.VBox([
    first_lc_empty_w,
    first_lc_loaded_w
    ])

display(widgets.HBox([left_box, right_box]))

In [ ]:
#@title Calculation
first_lc_empty = first_lc_empty_w.value
first_lc_loaded = first_lc_loaded_w.value

freq = get_freq(path_eigenfrequencies)
freq_empty = freq[(freq['LC'] < first_lc_loaded)].copy().reset_index()
freq_loaded = freq[(freq['LC'] >= first_lc_loaded)].copy().reset_index()
checked_freq = check_freq(freq_empty)
freq_to_calc = checked_freq[(checked_freq.vert_to_check == True) | (checked_freq.hor_to_check == True)]
print('')
print('ACCELERATIONS NEED TO BE CHECKED FOR THE FOLLOWING MODES:')
freq_to_calc


ACCELERATIONS NEED TO BE CHECKED FOR THE FOLLOWING MODES:


,index,LC,Title,f,vert_to_check,hor_to_check
0,0,7500,Eigenform 1 1.23 Hz,1.23,False,True
1,1,7501,Eigenform 2 1.38 Hz,1.38,False,True
2,2,7502,Eigenform 3 2.30 Hz,2.30,False,True
3,3,7503,Eigenform 4 2.34 Hz,2.34,False,True


# 2. Check accelerations

In [ ]:
#@title Inputs
software_scaling_factor_w = widgets.Dropdown(options=['Sofistik'], value='Sofistik')
bridge_type_w = widgets.Dropdown(options=[
    'Steel',
    'Steel-Concrete',
    'Reinforced Concrete',
    'Prestressed Concrete',
    'Wood'], value='Steel')
traffic_w = widgets.Dropdown(options=['Dense', 'Not Dense'], value='Dense')
vertical_w = widgets.Dropdown(options=['x', 'y', 'z'], value='z')
longitudinal_w = widgets.Dropdown(options=['x', 'y', 'z'], value='x')
lateral_w = widgets.Dropdown(options=['x', 'y', 'z'], value='y')

left_box = widgets.VBox([
    widgets.Label('Scaling factor'),
    widgets.Label('Bridge Type'),
    widgets.Label('Type of traffic'),
    widgets.Label('Vertical direction'),
    widgets.Label('Longitudinal direction'),
    widgets.Label('Transversal direction')
    ])

right_box = widgets.VBox([
    software_scaling_factor_w,
    bridge_type_w,
    traffic_w,
    vertical_w,
    longitudinal_w,
    lateral_w
    ])

display(widgets.HBox([left_box, right_box]))

In [ ]:
#@title Calculation
software_scaling_factor = software_scaling_factor_w.value
bridge_type = bridge_type_w.value
traffic = traffic_w.value
vertical = vertical_w.value
longitudinal = longitudinal_w.value
lateral = lateral_w.value

if software_scaling_factor == "Sofistik":
  factor_scaling = 0.001 / m.sqrt(1000)  # Scaling factor for Sofistik

nodes = get_nodes(path_node_coordinates)
Si = get_Si(path_unit_load_Si)
nodes = merge_nodes(nodes, Si)

modal_disp = get_modal_disp(path_modal_displacements)
modal_disp = scale_modal_disp(modal_disp, factor_scaling)
modal_disp_empty = modal_disp[(modal_disp['LC'] < first_lc_loaded)].copy().reset_index()
modal_disp_loaded = modal_disp[(modal_disp['LC'] >= first_lc_loaded)].copy().reset_index()
S_total = nodes.Si.sum()

n_lc = freq_to_calc['LC'].count()

results_mode = []
for mode_id in freq_to_calc.index.array:
    # Check mode
    mode = mode_id + 1
    mode_lc_empty = first_lc_empty + mode_id
    mode_lc_loaded = first_lc_loaded + mode_id
    mode_f_empty = freq_empty[freq_empty.LC == mode_lc_empty]['f'].item()
    mode_f_loaded = freq_loaded[freq_loaded.LC == mode_lc_loaded]['f'].item()
    # print(f'CHECK MODE {mode} - freq = {mode_f_empty} (empty), {mode_f_loaded} (loaded) :')
    results = pd.DataFrame(columns=['', 'vert', '', '', '', '', 'long', '', '', '', '', 'lat', '', '', ''])
    results.loc[0] = ['', 'TC 1', 'TC 2', 'TC 3', 'TC 4', '', 'TC 1', 'TC 2', 'TC 3', 'TC 4', '', 'TC 1', 'TC 2', 'TC 3', 'TC 4']
    for i in range(1, 15):
        results.loc[i] = ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
    results.iloc[1:, 0] = ['d (p/m2)',
                           'N (-)',
                           'N_lim (-)',
                           '<5% modal mass?',
                           'Modal mass',
                           'Load',
                           'Damping (%)',
                           'Neq (-)',
                           'gamma P (N)',
                           'q_eq',
                           'a_max',
                           'a_lim',
                           'is acceptable?',
                           'm_tmd'
                           ]

    index = 1
    for dir in ['vertical', 'longitudinal', 'lateral']:
        axis = get_axis(dir, vertical, longitudinal, lateral)
        for TC in range(1, 5):
            d = get_d(TC, S_total)
            N = d * S_total

            disp = modal_disp_empty[modal_disp_empty.LC == mode_lc_empty]
            # Check if pedestrian loads need to be considered
            mj = get_modal_mass(disp, axis)
            N_limit = 0.05 * mj / 70

            if N > N_limit:
                is_less_5pc = False
                load = 'loaded'
                disp = modal_disp_loaded[modal_disp_loaded.LC == mode_lc_loaded]
                mode_f = mode_f_loaded
            else:
                is_less_5pc = True
                load = 'empty'
                mode_f = mode_f_empty

            damping = get_damping(TC, bridge_type)
            N_eq = get_N_eq(N, damping, d)
            gp = get_gp(mode_f, dir)
            q_eq = N_eq / S_total * gp

            table_mode = merge_all_mode(disp, nodes)
            sum_si_disp_df = abs(table_mode[axis]) * table_mode['Si']
            sum_si_disp = sum_si_disp_df.sum()
            max_disp = abs(table_mode[axis]).max()
            a_max = q_eq * sum_si_disp / (2 * damping) * max_disp

            a_req = get_a_req(dir, TC, traffic)
            is_ok = a_max < a_req

            if is_ok:
                m_tmd = '-'
            else:
                a_new = a_max
                damping_new = damping
                while a_new > a_req:
                    damping_new += 0.00001
                    N_eq_new = get_N_eq(N, damping_new, d)
                    q_eq_new = N_eq_new / S_total * gp
                    a_new = q_eq_new * sum_si_disp / (2 * damping_new) * max_disp
                damping_eff = damping
                m_tmd = 0
                while damping_eff < damping_new:
                    m_tmd += 1
                    mu = m_tmd / mj
                    damping_eff = damping + (2 * m.sqrt(2 / (mu*(1+mu)))) ** -1

            results.iloc[1:, index] = [d, N, N_limit, is_less_5pc, mj, load, damping*100, N_eq, gp, q_eq, a_max, a_req, is_ok, m_tmd]

            index += 1
        index += 1

    results_mode.append(results)
    #print(results[:][:])
    #print('')

In [ ]:
#@title Output
mode_list = freq_to_calc.index.array + 1
mode_list_str = [str(x) for x in mode_list]
mose_list_int = [int(x) for x in mode_list]
mode_w = widgets.Dropdown(options=mode_list_str, value='1')

@widgets.interact(mode=mode_w)
def output(mode):
  mode_id = int(mode) - 1
  mode_lc_empty = first_lc_empty + mode_id
  mode_lc_loaded = first_lc_loaded + mode_id
  mode_f_empty = freq_empty[freq_empty.LC == mode_lc_empty]['f'].item()
  mode_f_loaded = freq_loaded[freq_loaded.LC == mode_lc_loaded]['f'].item()
  print(f'CHECK MODE {mode} - freq = {mode_f_empty} (empty), {mode_f_loaded} (loaded) :')
  return results_mode[mode_id]

interactive(children=(Dropdown(description='mode', options=('1', '2', '3', '4'), value='1'), Output()), _dom_c…